In [1]:
from datasets import load_dataset
dataset = load_dataset("cfilt/iitb-english-hindi")

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Using custom data configuration cfilt--iitb-english-hindi-911387c6837f8b91


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/85.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/500k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190M [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to C:\Users\Admin\.cache\huggingface\datasets\parquet\cfilt--iitb-english-hindi-911387c6837f8b91\0.0.0\1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
source_train_file = open("source_train.txt", "w+", encoding='utf8')
target_train_file = open("target_train.txt", "w+", encoding='utf8')
for translation_pair in dataset["train"]["translation"]:
    source_sentence = translation_pair["en"]
    target_sentence = translation_pair["hi"]
    source_train_file.write(source_sentence.strip("\n") + "\n")
    target_train_file.write(target_sentence.strip("\n") + "\n")
source_train_file.close()
target_train_file.close()

source_valid_file = open("source_valid.txt", "w+", encoding='utf8')
target_valid_file = open("target_valid.txt", "w+", encoding='utf8')
for translation_pair in dataset["validation"]["translation"]:
    source_sentence = translation_pair["en"]
    target_sentence = translation_pair["hi"]
    source_valid_file.write(source_sentence.strip("\n") + "\n")
    target_valid_file.write(target_sentence.strip("\n") + "\n")
source_valid_file.close()
target_valid_file.close()

source_test_file = open("source_test.txt", "w+", encoding='utf8')
target_test_file = open("target_test.txt", "w+", encoding='utf8')
for translation_pair in dataset["test"]["translation"]:
    source_sentence = translation_pair["en"]
    target_sentence = translation_pair["hi"]
    source_test_file.write(source_sentence.strip("\n") + "\n")
    target_test_file.write(target_sentence.strip("\n") + "\n")
source_test_file.close()
target_test_file.close()